### COMP3359 Artificial Intelligence Applications
Department of Computer Science, HKU
<br><br>

# <u>Checkpoint 1c - PyTorch</u>  


## Estimated Time to Finish:   
1~2 hours   

## Main Learning Objectives:   
-	Practise usage of common ML framework to construct simple application.

## Overview   
1.	[Introduction](#s1)  
2.	[Before You Start](#s2)
3.	[Preparation](#s3)
4.	[Task - Reuse Model and Make Predictions](#s4)
5.	[Submission](#s5)

-----


<a id=’s1’></a>
# 1 Introduction

It may be a good idea to kick-start our study of building AI applications by learning basic usage of some ML framework. This checkpoint extends our other material "ML Framework Learning Roadmap - PyTorch". The main task in this checkpoint is to reuse the model trained in the example in "ML Framework Learning Roadmap - PyTorch", and give predictions to data we provide.

-----   


<a id=’s2’></a>
# 2 Before You Start

## Referenced Tutorial

This checkpoint is mainly built by referencing the following tutorial. It is <b>strongly recommended</b> to study it once first to understand the context of this tutoiral.

Referenced tutorial:
- [PyTorch - TRAINING A CLASSIFIER](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py)

## Auxilliary Tools

In this checkpoint, you may need to use python packages to help you tackle the problems. <b>If you have no experience</b> using the following packages, it is <b>recommended</b> to check the following short tutorials and complete the simple exercises inside.

- NumPy
    - [NumPy UltraQuick Tutorial](https://colab.research.google.com/github/google/eng-edu/blob/master/ml/cc/exercises/numpy_ultraquick_tutorial.ipynb)
- Pandas
    - [Pandas DataFrame UltraQuick Tutorial](https://colab.research.google.com/github/google/eng-edu/blob/master/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)

Since in the context of AI, we often handles a large number of numerical values that are arraged as <b>(multi-dimensional) arrays</b> (e.g. vectors, matrices, tensors), you may pay attentation to the manipulations of such (multi-dimensional) arrays in these tutorials.

------

<a id=’s3’></a>
# 3 Preparation
"ML Framework Learning Roadmap - PyTorch" suggests a tutorial about training an image classifier model. To prepare ourselves for this checkpoint, we trained the same model here:



In [4]:
# Ref.: PyTorch - TRAINING A CLASSIFIER
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py
 
# Loading and normalizing CIFAR10
import torch
import torchvision
import torchvision.transforms as transforms
 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
 
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
 
print("===== Data =====")
print("# training samples: ", len(trainset))
print("# training batches: ", len(trainloader))
print("# test samples: ", len(testset))
print("# test batches: ", len(testloader))
print("(class label, class name): ")
print(list(zip(range(len(classes)), classes)))
 
 
# Define a Convolutional Neural Network
import torch.nn as nn
import torch.nn.functional as F
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
 
net = Net()
 
# Define a Loss function and optimizer
import torch.optim as optim
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
 
# Train the network
print("===== Model Training =====")
for epoch in range(2):  # loop over the dataset multiple times
 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
 
        # zero the parameter gradients
        optimizer.zero_grad()
 
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
 
print('Finished Training')
 
# Test the network on the test data
dataiter = iter(testloader)
images, labels = dataiter.next()
 
"""
# Evaluate on individual images
 
# print images
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
 
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
 
outputs = net(images)
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))
 
"""
# Evaluate on entire test dataset
print("===== Model Evaluation =====")
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
 
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
===== Data =====
# training samples:  50000
# training batches:  12500
# test samples:  10000
# test batches:  2500
(class label, class name): 
[(0, 'plane'), (1, 'car'), (2, 'bird'), (3, 'cat'), (4, 'deer'), (5, 'dog'), (6, 'frog'), (7, 'horse'), (8, 'ship'), (9, 'truck')]
===== Model Training =====
[1,  2000] loss: 2.193
[1,  4000] loss: 1.891
[1,  6000] loss: 1.712
[1,  8000] loss: 1.602
[1, 10000] loss: 1.514
[1, 12000] loss: 1.479
[2,  2000] loss: 1.406
[2,  4000] loss: 1.389
[2,  6000] loss: 1.359
[2,  8000] loss: 1.317
[2, 10000] loss: 1.295
[2, 12000] loss: 1.311
Finished Training
===== Model Evaluation =====
Accuracy of the network on the 10000 test images: 53 %


-----

<a id=’s4’></a>
# 4 Reuse Model and Make Predictions
In the previous section, we have trained a model for image recognition. Next, we will try to reuse the trained model and make predictions on the images we provide for this checkpoint.

We have prepared a data directory with few test images to be classified. The following sample codes assume the data directory is located next to this notebook, e.g.:
```
..
|-- Checkpoint1_PyTorch.ipynb
|-- images/
  |-- img0.jpg
  # more images…
```
If you put your data directory in somewhere else, you will need to modify the path to data directory accordingly below.

Our task is to <u><b> make predictions on the provided images using the trained model</b></u>. More specifically, we want to print out predictions as in:
```
# … all preceding steps.
# Print out predicted class names 
pred_class_names = [ classes[pl] for pl in pred_labels ]
print("Predictions: ", pred_class_names)
```
and your task here is to <u>complete the steps before printing out predictions</u>, which are briefly:
1.	Load image data.
2.	Preprocess data.
3.	Make predictions using trained model.

There are more than one way to carry out these steps and <u>you are welcomed to prepare predictions in your own fashion</u>. In case you are feeling uncertain about where to start, in the next code cell an example procedure is provided for you, and <u>you could complete the task by filling in the missing parts according to instructions given</u>. 

In addition, the following materials may help you to complete this task:
-	[Dataset Class - WRITING CUSTOM DATASETS, DATALOADERS AND TRANSFORMS]( https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class)
-	[How to read just one pic?](https://discuss.pytorch.org/t/how-to-read-just-one-pic/17434)



In [5]:
# Mount Google Drive for loading data if running in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
""" Reuse Trained Model and Make Prediction """
########################################################
# This is only a suggested method to make predictions  #
# on the provided data.                                #
#                                                      #
# You may modify or replace the following codes,       #
# as long as you can provide predictions from          #
# the trained model.                                   #
########################################################

# Specify data directory
#data_dir = "./images/"
data_dir = '/content/drive/MyDrive/images'

import torch
import skimage
from skimage import io

# Get image paths in data_dir
import os
data_paths = os.listdir(data_dir)
data_paths = [ os.path.join(data_dir, p) for p in data_paths ]
data_paths.sort()

##### Load Images #####
# Load all images into one big tensor
image_batch = []
for path in data_paths :
  print(path)
  # Load image file. You may try:
  #     skimage.io.imread: https://scikit-image.org/docs/dev/api/skimage.io.html?highlight=imread#skimage.io.imsave
  image = skimage.io.imread(path, as_gray=False)
  

  # Convert to PyTorch tensor. 
  # You may need to specify tensor as "float" type, which is required by our model.
  image_tensor = torch.Tensor(image)
  
  # As PyTorch Conv2d Layer operates on input with shape (N,C,H,W),
  # where N: batch size, C: #channels (e.g. RGB), H: height, W: width,
  # you may need to convert shape of image we just loaded from (H,W,C) to (C,H,W).
  # (see https://pytorch.org/docs/stable/nn.html#conv2d)
  # To change order of tensor dimensions, you may try: 
  #     torch.Tensor.permute: https://pytorch.org/docs/stable/nn.html#conv2d
  image_tensor = image_tensor.permute([2,0,1])
  
  # Store loaded image in our list
  image_batch.append(image_tensor) 

# Pack list of image tensors into one big tensor.
# (p.s. image_batch should has shape (B,C,H,W))
image_batch = torch.stack(image_batch)

##### Make Predictions using Trained Model #####
# Make predictions on images in 'image_batch' 

# Disable gradient update engine during evaluation phase 
with torch.no_grad() :
  # Make predictions using trained model
  #pred_labels = [net(images) for images in image_batch]
  _,pred_labels = torch.max(net(image_batch), 1)


############################################################
# Printing out predictions here is your goal of this task. #
############################################################
# Print out predicted class names
pred_class_names = [ classes[pl] for pl in pred_labels ]
print("Predictions: ", pred_class_names)

# Predictions:  ['ship', 'car', '?', 'truck', 'ship', '?', '?']

/content/drive/MyDrive/images/img0.jpg
/content/drive/MyDrive/images/img1.jpg
/content/drive/MyDrive/images/img2.jpg
/content/drive/MyDrive/images/img3.jpg
/content/drive/MyDrive/images/img4.jpg
/content/drive/MyDrive/images/img5.jpg
/content/drive/MyDrive/images/img6.jpg
Predictions:  ['ship', 'car', 'plane', 'truck', 'ship', 'plane', 'horse']


-----

<a id=’s5’></a>
# 5 Submission


To complete and submit your work, please submit the completed version of this notebook to Moodle. Please make sure that it can be executed without errors, and predictions from trained model are provided in clear, comprehensible fashion.


-----
